In [1]:
import cv2

from utils.loader import Loader
from utils.image import draw_around_box, draw_pose, get_area_of_interest
from utils.trainer import Trainer

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
# 1. Init loader, this will load the (non-image) dataset into memory
loader = Loader()
print(f'Dataset has {len(loader)} grasp attempts.')


# 2. Load an action and/or image
episode_index = 56
action = loader.get_action(episode_index, action_id=0)
print(f"Loaded action of episode {action['episode_id']} with reward: {action['reward']} at pose: {action['pose']}")
print(action)

rgbd_image = loader.get_image(episode_index, action_id=0, camera='rcd')

# You can also load action and images at once
action, rgb_image = loader.get_action(episode_index, action_id=0, images=['rc'])


# 3. Draw action and box on image for visualization
draw_around_box(rgbd_image, action['box_data'])
draw_pose(rgbd_image, action['pose'])
# 4. Get image area by an affine transformation. By specifying size_result, we can scale the final image down.
area = get_area_of_interest(rgbd_image, action['pose'], size_cropped=(200, 200), size_result=(150, 150))  # [px]
cv2.imshow('area', area)
cv2.waitKey(0)
print("3")
cv2.destroyAllWindows()


# 5. Iterate over all actions
for index, episode in enumerate(loader.yield_episodes()):
    print('Episode keys: ', list(episode.keys()))
    print('Action keys: ', list(episode['actions'][0].keys()))

    # loader.get_image(index, action_id=0, camera='rcd')
    break


# 6. Split into Training / Validation / Test set
# training_set, validation_set, test_set = Trainer.split(loader.episodes, seed=42)
# print(f'Training set length: {len(training_set)}')
# print(f'Validation set length: {len(validation_set)}')
# print(f'Test set length: {len(test_set)}')

Dataset has 22002 grasp attempts.
Loaded action of episode 2020-06-02-15-24-22-937 with reward: 0 at pose: {'x': 0.022446519796532102, 'y': -0.06984341434605564, 'z': -0.004660785129988632, 'a': -1.1223572697521091, 'b': -0.0005229883538433455, 'c': 0.00027451530815625347, 'd': 0.05}
{'pose': {'x': 0.022446519796532102, 'y': -0.06984341434605564, 'z': -0.004660785129988632, 'a': -1.1223572697521091, 'b': -0.0005229883538433455, 'c': 0.00027451530815625347, 'd': 0.05}, 'type': 'grasp', 'safe': 1, 'reward': 0, 'collision': True, 'estimated_reward': -1, 'method': 'Random', 'images': {'rd-v': {'info': {'pixel_size': 2000.0, 'min_depth': 0.2199999988079071, 'max_depth': 0.4099999964237213}, 'pose': {'x': -0.000301147668394397, 'y': -7.021008078000879e-05, 'z': 0.3499887353031891, 'a': -0.0005321141969116816, 'b': -0.00047430654395697047, 'c': -0.00035200678608271474}}, 'rc-v': {'info': {'pixel_size': 2000.0, 'min_depth': 0.2199999988079071, 'max_depth': 0.4099999964237213}, 'pose': {'x': -0

In [3]:
# Saving images and data into training_data_img list

data_img_d_small = []
# test_data_img = []
# validation_data_img = []

loader = Loader()
# training_set, validation_set, test_set = Trainer.split(loader.episodes, seed=42)

i = 0

for index, episode in tqdm(enumerate(loader.yield_episodes())):
    action = episode['actions'][0]

    id_ = Trainer.get_index(action)
    if id_ == 0:
        i += 1
    else:
        continue
    
    depth_img = loader.get_image(index, action_id=0, camera='rcd')
    draw_around_box(depth_img, action['box_data'])
    draw_pose(depth_img, action['pose'])
    area = get_area_of_interest(depth_img, action['pose'], size_cropped=(200, 200), size_result=(150, 150))
    area = cv2.cvtColor(area, cv2.COLOR_BGR2GRAY)
    area = cv2.normalize(area, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    data_img_d_small.append([area, action['reward']])
#     break
# cv2.imshow('area', area)
# cv2.waitKey(0)
# print("works")
# cv2.destroyAllWindows()

22002it [00:49, 440.79it/s] 


In [4]:
i

4845

In [5]:
print(len(data_img_d_small))

4845


In [6]:
d = np.array(data_img_d_small)
np.random.shuffle(d)
d

array([[array([[ 54,  29,  29, ...,  33,  33,  29],
       [ 29,  29,  29, ...,  30,  29,  29],
       [ 29,  29,  42, ...,  52,  45,  54],
       ...,
       [ 29,  30,  35, ...,  69,  82, 123],
       [ 29,  30,  37, ...,  71,  73,  60],
       [ 29,  29,  30, ...,  70,  74,  56]], dtype=uint8),
        1.0],
       [array([[ 29,  29,  29, ...,  29,  29,  29],
       [ 29,  29,  56, ...,  91,  98,  83],
       [ 29,  29,  96, ..., 122, 118, 103],
       ...,
       [ 33,  29,  77, ...,  53,  53,  53],
       [ 36,  29,  66, ...,  53,  53,  53],
       [ 29,  29,  47, ...,  53,  53,  53]], dtype=uint8),
        0],
       [array([[ 30,  30,  30, ...,  30,  30,  30],
       [ 30,  27,  21, ...,  19,  25,  23],
       [ 30,  26,  24, ...,  18,  19,  27],
       ...,
       [ 30,  44, 122, ...,  61,  60,  30],
       [ 30,  71,  63, ...,  53,  58,  40],
       [ 30,  30,  30, ...,  30,  30,  32]], dtype=uint8),
        0],
       ...,
       [array([[49, 49, 49, ..., 49, 49, 49],
       

In [7]:
train_small, validate_small, test_small = np.split(d, [int(.6*len(d)), int(.8*len(d))])

In [8]:
print(len(train_small),len(test_small),len(validate_small))

2907 969 969


In [9]:
data_img_d_med = []

loader = Loader()

i = 0

for index, episode in tqdm(enumerate(loader.yield_episodes())):
    action = episode['actions'][0]

    id_ = Trainer.get_index(action)
    if id_ == 1:
        i += 1
    else:
        continue
    
    depth_img = loader.get_image(index, action_id=0, camera='rcd')
    draw_around_box(depth_img, action['box_data'])
    draw_pose(depth_img, action['pose'])
    area = get_area_of_interest(depth_img, action['pose'], size_cropped=(200, 200), size_result=(150, 150))
    area = cv2.cvtColor(area, cv2.COLOR_BGR2GRAY)
    area = cv2.normalize(area, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    data_img_d_med.append([area, action['reward']])

22002it [00:52, 421.96it/s]


In [10]:
m = np.array(data_img_d_med)
np.random.shuffle(m)
m

array([[array([[51, 51, 51, ..., 51, 51, 51],
       [51, 68, 77, ..., 50, 43, 51],
       [51, 80, 92, ...,  0,  0, 51],
       ...,
       [51, 92, 92, ..., 91, 83, 52],
       [51, 92, 92, ..., 89, 84, 51],
       [51, 59, 54, ..., 76, 69, 51]], dtype=uint8),
        0],
       [array([[43, 45, 51, ..., 43, 43, 43],
       [43, 57, 77, ..., 77, 77, 51],
       [43, 52, 77, ..., 77, 77, 57],
       ...,
       [43, 72, 80, ..., 68, 72, 43],
       [43, 56, 59, ..., 65, 65, 43],
       [43, 43, 43, ..., 43, 43, 43]], dtype=uint8),
        0],
       [array([[37, 37, 37, ..., 37, 37, 37],
       [37, 37, 49, ..., 62, 41, 45],
       [37, 37, 55, ..., 66, 66, 66],
       ...,
       [37, 37, 33, ..., 36, 39, 40],
       [37, 37, 33, ..., 34, 39, 43],
       [37, 37, 36, ..., 35, 38, 39]], dtype=uint8),
        1.0],
       ...,
       [array([[42, 42, 42, ..., 42, 42, 42],
       [42, 42, 46, ..., 60, 50, 42],
       [42, 46, 37, ..., 76, 66, 42],
       ...,
       [42, 70, 72, ..., 63

In [11]:
train_med, validate_med, test_med = np.split(m, [int(.6*len(m)), int(.8*len(m))])

In [12]:
data_img_d_large = []

loader = Loader()

i = 0

for index, episode in tqdm(enumerate(loader.yield_episodes())):
    action = episode['actions'][0]

    id_ = Trainer.get_index(action)
    if id_ == 2:
        i += 1
    else:
        continue
    
    depth_img = loader.get_image(index, action_id=0, camera='rcd')
    draw_around_box(depth_img, action['box_data'])
    draw_pose(depth_img, action['pose'])
    area = get_area_of_interest(depth_img, action['pose'], size_cropped=(200, 200), size_result=(150, 150))
    area = cv2.cvtColor(area, cv2.COLOR_BGR2GRAY)
    area = cv2.normalize(area, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    data_img_d_large.append([area, action['reward']])

22002it [01:01, 359.32it/s]


In [13]:
l = np.array(data_img_d_med)
np.random.shuffle(l)
l

array([[array([[ 45,  54,  43, ...,  43,  43,  43],
       [ 43,  43,  43, ...,  48,  43,  55],
       [ 43,  61,  61, ...,  94,  92,  62],
       ...,
       [ 43, 139, 137, ...,  76,  76,  43],
       [ 43, 126, 160, ...,  76,  71,  43],
       [ 43, 172, 145, ...,  76,  66,  43]], dtype=uint8),
        1.0],
       [array([[ 76,  75,  75, ...,  75,  75,  75],
       [ 75,  94,  75, ...,  85,  75,  88],
       [ 75, 101, 134, ..., 150, 138, 113],
       ...,
       [ 75,  75, 222, ..., 159, 149,  75],
       [ 75, 104, 218, ..., 147, 142, 105],
       [ 83,  75, 224, ..., 140, 136,  98]], dtype=uint8),
        0],
       [array([[29, 29, 29, ..., 29, 29, 29],
       [29, 31, 31, ..., 29, 29, 29],
       [29, 31, 27, ..., 24, 25, 29],
       ...,
       [29, 41, 62, ..., 42, 44, 33],
       [29, 38, 65, ..., 37, 36, 32],
       [29, 30, 31, ..., 35, 34, 31]], dtype=uint8),
        1.0],
       ...,
       [array([[31, 31, 31, ..., 31, 31, 31],
       [31, 31, 13, ..., 36, 30, 31],
   

In [14]:
train_large, validate_large, test_large = np.split(l, [int(.6*len(l)), int(.8*len(l))])

In [15]:
data_img_d_xlarge = []

loader = Loader()

i = 0

for index, episode in tqdm(enumerate(loader.yield_episodes())):
    action = episode['actions'][0]

    id_ = Trainer.get_index(action)
    if id_ == 3:
        i += 1
    else:
        continue
    
    depth_img = loader.get_image(index, action_id=0, camera='rcd')
    draw_around_box(depth_img, action['box_data'])
    draw_pose(depth_img, action['pose'])
    area = get_area_of_interest(depth_img, action['pose'], size_cropped=(200, 200), size_result=(150, 150))
    area = cv2.cvtColor(area, cv2.COLOR_BGR2GRAY)
    area = cv2.normalize(area, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    data_img_d_xlarge.append([area, action['reward']])

22002it [00:59, 369.81it/s]


In [16]:
xl = np.array(data_img_d_med)
np.random.shuffle(xl)
xl

array([[array([[ 30,  30,  30, ...,  30,  30,  30],
       [ 30,  34,  39, ...,  48,  45,  33],
       [ 30,  38,  46, ...,  54,  54,  34],
       ...,
       [ 30,  37,  76, ...,   9,   0,  18],
       [ 30,  50, 105, ...,  66,   6,  21],
       [ 30,  30,  37, ...,  30,  30,  29]], dtype=uint8),
        0],
       [array([[ 30,  30,  30, ...,  30,  30,  30],
       [ 30,  30,  34, ...,  24,  24,  30],
       [ 30,  32,  58, ...,  22,  21,  30],
       ...,
       [ 30,  96, 133, ...,  46,  44,  30],
       [ 30,  83, 129, ...,  45,  42,  31],
       [ 30,  67, 121, ...,  38,  34,  30]], dtype=uint8),
        0],
       [array([[103,  60,  60, ...,  60,  60,  60],
       [ 89,  63, 125, ..., 158, 127, 163],
       [ 60,  60, 190, ..., 176, 173, 129],
       ...,
       [ 60,  62,  78, ...,  78,  79,  78],
       [ 60,  60,  77, ...,  78,  78,  73],
       [ 60,  66,  71, ...,  61,  60,  60]], dtype=uint8),
        0],
       ...,
       [array([[ 48,  44,  44, ...,  44,  44,  44],
   

In [17]:
train_xlarge, validate_xlarge, test_xlarge = np.split(xl, [int(.6*len(xl)), int(.8*len(xl))])

In [18]:
# when need to save
filename = 'training_data_d_0.025'
outfile = open(filename,'wb')
pickle.dump(train_small,outfile)
outfile.close()

filename = 'test_data_d_0.025'
outfile = open(filename,'wb')
pickle.dump(test_small,outfile)
outfile.close()

filename = 'validate_data_d_0.025'
outfile = open(filename,'wb')
pickle.dump(validate_small,outfile)
outfile.close()

filename = 'training_data_d_0.05'
outfile = open(filename,'wb')
pickle.dump(train_med,outfile)
outfile.close()

filename = 'test_data_d_0.05'
outfile = open(filename,'wb')
pickle.dump(test_med,outfile)
outfile.close()

filename = 'validate_data_d_0.05'
outfile = open(filename,'wb')
pickle.dump(validate_med,outfile)
outfile.close()

filename = 'training_data_d_0.07'
outfile = open(filename,'wb')
pickle.dump(train_large,outfile)
outfile.close()

filename = 'test_data_d_0.07'
outfile = open(filename,'wb')
pickle.dump(test_large,outfile)
outfile.close()

filename = 'validate_data_d_0.07'
outfile = open(filename,'wb')
pickle.dump(validate_large,outfile)
outfile.close()

filename = 'training_data_d_0.086'
outfile = open(filename,'wb')
pickle.dump(train_xlarge,outfile)
outfile.close()

filename = 'test_data_d_0.086'
outfile = open(filename,'wb')
pickle.dump(test_xlarge,outfile)
outfile.close()

filename = 'validate_data_d_0.086'
outfile = open(filename,'wb')
pickle.dump(validate_xlarge,outfile)
outfile.close()
